## Run a batch of videos

This code runs DVT on all the videos in a particular directory and produces a website output in a folder called full-data-output

In [ ]:
import os

directory = "clips/trailers/"

for file in os.listdir(directory):
    if file.endswith(".mp4"):
        print("Running file: " + file)
        !python -m dvt video-viz {directory + file} --dirout full-data-output
        

In [ ]:
!python -m dvt video-viz clips/trailers/Official_Trailer_TROUBLE_WITH_THE_CURVE-Clint_Eastwood_Justin_Timberlake.mp4 --dirout full-data-output
!python -m dvt video-viz clips/trailers/Terminator_Dark_Fate.mp4  --dirout full-data-output
!python -m dvt video-viz clips/trailers/The_Terminator_1984_Official_Trailer_Arnold_Schwarzenegger.mp4  --dirout full-data-output
!python -m dvt video-viz clips/protest/anti_vietnam_war_protest.mp4  --dirout full-data-output


## Compile JSON data to CSV
To make it easy to read the numeric outputs from DVT, this code reads the JSON outout for every video and pulls some potentially interesting features together into one CSV. It draws video name, number of shots, number of shots with people, objects detected, and people detected more or less straight from the JSON; it calculates the avgerage number of people per shot, most common shot length, and average shot duration in seconds.

In [19]:
import csv, json, os
from statistics import mode

# Data directory containing a JSON file for each video
directory = "full-data-output/data/"

# Output will be written to AllOutputs.csv
with open("AllOutputs.csv", 'w') as outfile:
    writer = csv.writer(outfile, delimiter=",") 
    
    # Write header row
    writer.writerow(["Video name", "Num shots", "Num shots with people", "Avg people per shot", "Most common shot length",
                            "Avg shot duration (in seconds)", "Objects detected", "People detected"])
    
    # Loop over JSON files
    for file in os.listdir(directory):
        if file != "toc.json": # skip the table of contents file
            with open(directory + file, 'r') as videojson:
                data = json.load(videojson)
                name = data["meta"][0]["input_filename"]
                num_shots = len(data["cut"])


                # Find most common shot length
                shot_lengths = []
                for l in data["length"]:
                    shot_lengths.append(l["shot_length"])
                try:
                    mode_shot_length = mode(shot_lengths)
                except:
                    mode_shot_length = ""


                # Counting people and objects
                checked_frames = []
                num_shots_peopled = 0
                objects = []
                people_per_shot = {}
                for obj in data["obj"]:
                    if obj["frame"] in checked_frames:
                        if obj["category"] == "person":
                            people_per_shot[obj["frame"]] += 1
                    else:
                        # Count frames with people
                        if obj["category"] == "person":
                            num_shots_peopled += 1
                            people_per_shot[obj["frame"]] = 1
                        else:
                            people_per_shot[obj["frame"]] = 0
                        checked_frames.append(obj["frame"])

                    # Add objects
                    if obj["category"] != "person" and obj["category"] not in objects:
                        objects.append(obj["category"])

                avg_people_per_shot = sum(people_per_shot.values()) / len(people_per_shot.values())

                # Faces detected
                people = []
                if "people" in data:
                    for person in data["people"]:
                        if person["person"] not in people:
                            people.append(person["person"])

                # Calculate average shot length in seconds
                shot_durations = []
                for shot in data["cut"]:
                    shot_durations.append(shot["frame_end"]-shot["frame_start"])
                avg_shot_duration = (sum(shot_durations)/len(shot_durations))/data["meta"][0]["fps"]


                writer.writerow([name, num_shots, num_shots_peopled, avg_people_per_shot, mode_shot_length,
                            avg_shot_duration, '|'.join(objects), '|'.join(people)])             
                

## Add face data to output site

In [18]:
# Generate data site
!python -m dvt video-viz face-recognition-demo/Speed_Clip_Faces.mp4 --path-to-faces=face-recognition-demo/faces/speed/clip_faces --dirout face-recognition-demo/dvt-output-data
!python -m dvt video-viz face-recognition-demo/Speed_Hard_Faces.mp4 --path-to-faces=face-recognition-demo/faces/speed/hard_faces --dirout face-recognition-demo/dvt-output-data
!python -m dvt video-viz face-recognition-demo/Speed_Outside_Faces.mp4 --path-to-faces=face-recognition-demo/faces/speed/outside_faces --dirout face-recognition-demo/dvt-output-data

Using TensorFlow backend.
tracking <tf.Variable 'Variable:0' shape=(9, 4) dtype=float32> anchors
tracking <tf.Variable 'Variable_1:0' shape=(9, 4) dtype=float32> anchors
tracking <tf.Variable 'Variable_2:0' shape=(9, 4) dtype=float32> anchors
tracking <tf.Variable 'Variable_3:0' shape=(9, 4) dtype=float32> anchors
tracking <tf.Variable 'Variable_4:0' shape=(9, 4) dtype=float32> anchors
Processing annotators:  |#######################         | 11/15Killed
Using TensorFlow backend.
tracking <tf.Variable 'Variable:0' shape=(9, 4) dtype=float32> anchors
tracking <tf.Variable 'Variable_1:0' shape=(9, 4) dtype=float32> anchors
tracking <tf.Variable 'Variable_2:0' shape=(9, 4) dtype=float32> anchors
tracking <tf.Variable 'Variable_3:0' shape=(9, 4) dtype=float32> anchors
tracking <tf.Variable 'Variable_4:0' shape=(9, 4) dtype=float32> anchors
Processing annotators:  |################################| 15/15
Progress: Killed
Using TensorFlow backend.
tracking <tf.Variable 'Variable:0' shape=(9

In [16]:
import json, cv2, os
from matplotlib import pyplot as plt

print(os.getcwd())
directory = "face-recognition-demo/dvt-output-data/"
with open(directory + "data/Speed_Outside_Faces.json", 'r') as file:
    videojson = json.load(file)

people = videojson["people"]

filename = videojson["meta"][0]["input_filename"]
outdir = directory + "img/" + filename + "/faces"
os.mkdir(outdir)

for person in people:
    frame_num = person["frame"]
    frame_img = directory + "img/" + filename + "/frames/frame-" + str(frame_num).zfill(6) + ".png"
    img = cv2.imread(frame_img)
    cv2.rectangle(img, (person["left"], person["top"]), (person["right"], person["bottom"]), (24, 127, 245), 3)
    cv2.putText(img, person["person"], (person["left"], person["bottom"] + 25), cv2.FONT_HERSHEY_SIMPLEX, 1, (24, 127, 245))

    print(outfile)
    cv2.imwrite(outdir + "/frame-" + str(frame_num).zfill(6) + ".png", img)
    #cv2.imshow("img", img)
    plt.imshow(img)
    

/sharedfolder


FileNotFoundError: [Errno 2] No such file or directory: 'face-recognition-demo/dvt-output-data/data/Speed_Outside_Faces.json'